In [1]:
#%cd drive/MyDrive/Colab_Notebooks/Observation-earth/

In [2]:
#!git clone https://github.com/louisPoulain/IPEO-Understanding-the-Amazon-from-Space.git

In [3]:
#%cd IPEO-Understanding-the-Amazon-from-Space/

In [4]:
#!git checkout Test_Louis

In [5]:
#!pip install -q -U pytorch_lightning tensorboard

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
%cd drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space/
!git checkout Test_Louis
!git pull

/content/drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space
M	main_louis.ipynb
Already on 'Test_Louis'
Your branch is up to date with 'origin/Test_Louis'.
Already up to date.


In [8]:
!pip install -q -U pytorch_lightning tensorboard
!pip install pandas==1.5.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires tensorboard<2.10,>=2.9, but you have tensorboard 2.11.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 91.1 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [9]:
from dataset import DatasetAmazon
from model import PlanetModel, testModel, ResNet
import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
import time

from pytorch_lightning.callbacks import ModelCheckpoint
import os
import sys
#sys.path.append("drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space/")
dir_path = "drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO-Understanding-the-Amazon-from-Space"

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
full, tiny = True, False
train_dataset = DatasetAmazon(full=full, tiny=tiny, path_to_labels="CSV/train_label_vector.pkl")
val_dataset = DatasetAmazon(full=full, tiny=tiny, val=True, path_to_labels="CSV/train_label_vector.pkl")
print(len(train_dataset), len(val_dataset))

loading took  0.9203145503997803
loading took  0.22541546821594238
24285 6064


In [12]:
pretrained = False
depth = 18
test_model = ResNet(depth=depth) if pretrained else testModel(max_channels=512) 
model = PlanetModel(model=test_model)

In [13]:
csv_logger = pl_loggers.CSVLogger(save_dir="", name = "logs")

#checkpoint_callback = ModelCheckpoint(
#    dirpath="checkpoints",#dir_path + 
#    filename='test-{epoch}-{val_accuracy:.2f}',
#    monitor="val_accuracy",#val_loss
#    mode="max"#min
#    )

max_epochs = 50

checkpoint_callback = ModelCheckpoint(
    dirpath="../IPEO_Planet_project/checkpoints",
    filename=f'ResNet{depth}'+'-{epoch}_'+f'over_{max_epochs}'+'-{val_accuracy:.2f}',
    monitor="val_accuracy",
    mode="max"
    ) if pretrained else ModelCheckpoint(
    dirpath="../IPEO_Planet_project/checkpoints",
    filename='test-{epoch}_'+ f'over_{max_epochs}'+ '-{val_accuracy:.2f}',
    monitor="val_accuracy",
    mode="max"
    )

In [14]:
trainer = pl.Trainer(max_epochs=max_epochs, accelerator="gpu", devices=[0], 
                     logger=csv_logger, callbacks=[checkpoint_callback],
                     resume_from_checkpoint=None, check_val_every_n_epoch=1)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [15]:
batch_size = 128 if full else (3 if tiny else 20)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, persistent_workers=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, persistent_workers=True)

In [16]:
t = time.time()
trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=val_loader)
training_time = time.time()-t
print(f"Training time: {training_time}s")

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /content/drive/MyDrive/Colab_Notebooks/Observation-earth/IPEO_Planet_project/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type        | Params
--------------------------------------
0 | model | testModel   | 9.5 M 
1 | loss  | custom_loss | 0     
--------------------------------------
9.5 M     Trainable params
0         Non-trainable params
9.5 M     Total params
37.829    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


Training time: 9302.300100326538s


In [17]:
import matplotlib.pyplot as plt

In [18]:
#from accuracy_metrics import Hamming_distance, transform_pred, overall_acc
#test_dataset = DatasetAmazon(full=full, test=True, path_to_labels="CSV/train_label_vector.pkl")
#test_dl = torch.utils.data.DataLoader(test_dataset, batch_size = 1, num_workers=4, shuffle=False)
#print(len(test_dataset))

# replace with the checkpoint of your trained model
#model_name = "ResNet18-epoch=47_over_50-val_accuracy=0.97.ckpt"
#checkpoint_path = "../IPEO_Planet_project/checkpoints/"+model_name

#rs = trainer.predict(model, dataloaders=test_dl)

#y_hat, y = list(map(list, zip(*rs)))
#overall_accuracy = 0.0
#hamming_dist = 0.0
#for i in range(len(y_hat)):
#    overall_accuracy += overall_acc(y_hat[i], y[i])
#    hamming_dist += Hamming_distance(y_hat[i], y[i])
#overall_accuracy = overall_accuracy/len(y_hat)
#hamming_dist = hamming_dist/len(y_hat)
                                        
#print(f"Overall accuracy: {overall_accuracy:.4f}\nHamming distance: {hamming_dist}")